In [15]:
import numpy as np
import random

# set random seed for reproducablitliy 
np.random.seed(12345678) 
random.seed(12345678)

### Define functions for randomizations

In [16]:
def get_list_encoded_Fs(n_observed_TTT, n_observed_TTC):
    list_of_encoded_Fs = ['TTT']*int(n_observed_TTT) + ['TTC']*int(n_observed_TTC)
    return list(np.random.permutation(list_of_encoded_Fs))

In [17]:
def run_randomization(list_of_encoded_Fs_shuffled, FF_num):

    n_randomiziations = 1000

    random_encoded_n_TTTTTT = []

    for n in range(n_randomiziations): 
        temp_list_of_encoded_Fs_shuffled = list_of_encoded_Fs_shuffled.copy()
        random_encoded_FF = []
        for n_FF in range(int(FF_num)): 
            idx = random.randrange(0, len(temp_list_of_encoded_Fs_shuffled))
            random_F_1 = temp_list_of_encoded_Fs_shuffled.pop(idx)

            idx = random.randrange(0, len(temp_list_of_encoded_Fs_shuffled))
            random_F_2 = temp_list_of_encoded_Fs_shuffled.pop(idx)

            # add to list
            random_encoded_FF.append(random_F_1+random_F_2) 

        unique_counts = np.unique(random_encoded_FF, return_counts=True)
        try:
            idx_TTTTTT=list(unique_counts[0]).index('TTTTTT')
            random_encoded_n_TTTTTT.append(list(unique_counts[1])[idx_TTTTTT])
        except ValueError:
            random_encoded_n_TTTTTT.append(0)
            
    return random_encoded_n_TTTTTT


In [18]:
def compute_p(random_encoded_n_TTTTTT, TTTTTT_observed, n_randomiziations):
    return float(1-2*abs((1.0*sum(np.array(random_encoded_n_TTTTTT)>TTTTTT_observed)/n_randomiziations)-0.5))

### import observed FF, TTC, TTT values in different viral genomes

In [28]:
import pandas as pd

In [32]:
fname = '../../resources/Supplementary_Table_4_FF_codon_frequencies_genomes.csv'
df = pd.read_csv(fname)
df['virus'] = df['Unnamed: 0']

aa_num	Total number of aminoacids 	
F_num	Total number of F	
TTC_num	how many of those are codified by TTC	
TTT_num	how many of those are codified by TTT	
FF_num	Total number of 2 F in a row	
TTCTTT_num	number of FF codified by that codon combination	
TTTTTT_num		
TTCTTC_num		
TTC_freq	codon frequency (TTC_num/F_num)	
TTT_freq		
TTCTTT_freq	codon frequency (TTCTTT_num/(FF_num)	
TTTTTT_freq		
TTCTTC_freq		
prob_TTCTTT	Probability of finding FF codified by that codon combination (TTC_freq*TTT_freq)	
genome_wide_probTTCTTT	Probability of finding FF codified by that codon combination along the genome (prob_TTCTTT^FF_num)

In [35]:
df.columns

Index(['Unnamed: 0', 'ref_genome', 'aa_num', 'F_num', 'TTC_num', 'TTT_num',
       'FF_num', 'TTCTTT_num', 'TTTTTT_num', 'TTCTTC_num', 'TTC_freq',
       'TTT_freq', 'TTCTTT_freq', 'TTTTTT_freq', 'TTCTTC_freq', 'virus type',
       'virus'],
      dtype='object')

In [37]:
df = df[['virus', 'ref_genome', 'TTC_num', 'TTT_num', 'FF_num', 'TTTTTT_num', 'virus type']]

In [38]:
df

,virus,ref_genome,TTC_num,TTT_num,FF_num,TTTTTT_num,virus type
0,Drosophila C virus,NC_001834.1,30,95,4,0,RNA virus
1,Cricket paralysis virus,NC_003924.1,34,86,6,1,RNA virus
2,Nora virus,NC_007919.3,35,73,3,1,RNA virus
3,Sigma virus,JX403934.1,89,66,4,0,RNA virus
4,Acute bee paralysis virus,NC_002548.1,35,87,5,0,RNA virus
5,deformed wing virus,NC_004830.2,29,83,4,0,RNA virus
6,SarsCoV2,NC_045512.2,149,351,18,1,RNA virus
7,Human simplex viris 2,NC_001798.2,744,613,56,11,DNA virus
8,human cytomegalovirus,OK000912.1,1132,1092,116,28,DNA virus
9,Kallithea virus,NC_033829.1,435,1089,43,24,DNA virus


In [39]:
def run_all(row):
    n_randomiziations = 1000
    list_of_encoded_Fs_shuffled = get_list_encoded_Fs(row["TTT_num"], row["TTC_num"])
    random_encoded_n_TTTTTT = run_randomization(list_of_encoded_Fs_shuffled, row["FF_num"])
    
    mean_TTTTTT_randomizations = np.mean(random_encoded_n_TTTTTT)
    quantil_TTTTTT_randomizations25 = sorted(random_encoded_n_TTTTTT)[int(n_randomiziations*0.025)]
    quantil_TTTTTT_randomizations75 = sorted(random_encoded_n_TTTTTT)[-int(n_randomiziations*0.025)]
    p  = compute_p(random_encoded_n_TTTTTT, row["TTTTTT_num"], n_randomiziations)
    
    return mean_TTTTTT_randomizations, quantil_TTTTTT_randomizations25, quantil_TTTTTT_randomizations75, p
    
    

In [40]:
df[["mean_TTTTTT_rand", 
    "25quantil_TTTTTT_rand", 
    "75quantil_TTTTTT_rand",
    "p-value"]] = df.apply(lambda row: run_all(row), axis='columns', result_type='expand')

/var/folders/dw/8dl4p6h53cgcmk6cf_09v5f40000gr/T/ipykernel_47129/3327794645.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["mean_TTTTTT_rand",
/var/folders/dw/8dl4p6h53cgcmk6cf_09v5f40000gr/T/ipykernel_47129/3327794645.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["mean_TTTTTT_rand",
/var/folders/dw/8dl4p6h53cgcmk6cf_09v5f40000gr/T/ipykernel_47129/3327794645.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [41]:
df

,virus,ref_genome,TTC_num,TTT_num,FF_num,TTTTTT_num,virus type,mean_TTTTTT_rand,25quantil_TTTTTT_rand,75quantil_TTTTTT_rand,p-value
0,Drosophila C virus,NC_001834.1,30,95,4,0,RNA virus,2.328,1.0,4.0,0.046
1,Cricket paralysis virus,NC_003924.1,34,86,6,1,RNA virus,3.051,1.0,5.0,0.182
2,Nora virus,NC_007919.3,35,73,3,1,RNA virus,1.418,0.0,3.0,0.904
3,Sigma virus,JX403934.1,89,66,4,0,RNA virus,0.687,0.0,2.0,0.932
4,Acute bee paralysis virus,NC_002548.1,35,87,5,0,RNA virus,2.542,1.0,5.0,0.046
5,deformed wing virus,NC_004830.2,29,83,4,0,RNA virus,2.187,0.0,4.0,0.070
6,SarsCoV2,NC_045512.2,149,351,18,1,RNA virus,8.817,5.0,13.0,0.002
7,Human simplex viris 2,NC_001798.2,744,613,56,11,DNA virus,11.344,6.0,17.0,0.924
8,human cytomegalovirus,OK000912.1,1132,1092,116,28,DNA virus,28.056,20.0,37.0,0.918
9,Kallithea virus,NC_033829.1,435,1089,43,24,DNA virus,21.926,15.0,28.0,0.422


In [42]:
df.to_csv("Supplementary_Table_4.csv")